# Optimization

### optimizer的工作方式
在training loop中，优化会通过三步完成：
1. 调用optimizer.zero_grad()来重置参数的梯度值。之所以要重置，是因为gradient的计算是累加的，为了防止double-counting，在每次迭代前要将初始值归0
2. 调用loss.backward()来启动loss的反向传播。pytorch会存储每个parameter相对loss的梯度值
3. 调用optimizer.step()，用backward得到的所有参数的梯度值来更新parameter

In [2]:
# 先构建model
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

# training set
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

# test set
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

# Load data
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

# construct a NN model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

# instantiate a model
model = NeuralNetwork()

/home/roark/anaconda3/envs/231n/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/roark/anaconda3/envs/231n/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [3]:
# set hyper parameters
learning_rate = 1e-3
batch_size = 64
epochs = 5

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

In [5]:
# Initialize the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [6]:
# 定义train_loop
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [7]:
# 定义test loop
def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.297951  [   64/60000]
loss: 2.293491  [ 6464/60000]
loss: 2.276759  [12864/60000]
loss: 2.271200  [19264/60000]
loss: 2.256204  [25664/60000]
loss: 2.230252  [32064/60000]
loss: 2.239081  [38464/60000]
loss: 2.211486  [44864/60000]
loss: 2.212490  [51264/60000]
loss: 2.174179  [57664/60000]
Test Error: 
 Accuracy: 50.7%, Avg loss: 2.172608 

Epoch 2
-------------------------------
loss: 2.182818  [   64/60000]
loss: 2.171914  [ 6464/60000]
loss: 2.120591  [12864/60000]
loss: 2.129127  [19264/60000]
loss: 2.088525  [25664/60000]
loss: 2.031816  [32064/60000]
loss: 2.055591  [38464/60000]
loss: 1.988626  [44864/60000]
loss: 2.002173  [51264/60000]
loss: 1.910487  [57664/60000]
Test Error: 
 Accuracy: 61.5%, Avg loss: 1.917411 

Epoch 3
-------------------------------
loss: 1.952132  [   64/60000]
loss: 1.917939  [ 6464/60000]
loss: 1.806538  [12864/60000]
loss: 1.833902  [19264/60000]
loss: 1.743233  [25664/60000]
loss: 1.676725  [32064/600